# SMA Crossover Strategy with Martingale
Author: Mohamed Abbas El kayal

## Disclaimer
Trading the financial markets imposes a risk of financial loss. TraderPy is not responsible for any financial losses that viewers suffer. Content is educational only and does not serve as financial advice. Information or material is provided ‘as is’ without any warranty. 

Past trading results do not indicate future performance. Strategies that worked in the past may not reflect the same results in the future.

In [1]:
import datetime
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np

from datetime import datetime
from IPython.display import display

In [2]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [3]:
# connect to MetaTrader5 as mt5
mt5.initialize()

True

In [4]:
# settings
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_H1
start_pos = 0
num_bars = 6500

fast_sma_period = 50
slow_sma_period = 100

In [5]:
# Requesting historical data from MetaTrader 5
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)

# Converting bars we got from MetaTrader5 into dataframe
df = pd.DataFrame(bars)[['time', 'open', 'high', 'low', 'close', 'spread']]

#  Re-factoring time format into human readable
df['time'] = pd.to_datetime(df['time'], unit='s')


In [6]:
# Visualize Close Price
px.line(df, x='time', y='close')

In [7]:
#  define the indicators
def indicator_sma (dataframe):
    dataframe['fast_sma'] = dataframe['close'].rolling(fast_sma_period).mean()
    dataframe['slow_sma'] = dataframe['close'].rolling(slow_sma_period).mean()

    # finding crossovers
    dataframe['prev_fast_sma'] = dataframe['fast_sma'].shift(1)

    dataframe.dropna(inplace=True)

indicator_sma(dataframe=df)

In [8]:
def create_signal(dataframe):
    
    conditions_signal = [
        # dataframe['fast_sma'].gt(dataframe['slow_sma']) & dataframe['prev_fast_sma'].lt(dataframe['slow_sma']), # Buy Signal
        # dataframe['fast_sma'].lt(dataframe['slow_sma']) & dataframe['prev_fast_sma'].gt(dataframe['slow_sma']), # Sell Signal
        dataframe['fast_sma'].gt(dataframe['slow_sma']), # up signal
        dataframe['fast_sma'].lt(dataframe['slow_sma']), # down signal
    ]

    choices_signal = [
        # 'buy',
        # 'sell', 
        'buy', 
        'sell'
        ]

    dataframe['signal'] = np.select(conditions_signal, choices_signal, default=None)

    # export dataframe to excel file

    # create file name based on current date and time
    # filename = datetime.now().strftime('%Y-%m-%d-%H-%M-%S.xlsx')

    # export dataframe to excel file
    # df.to_excel(filename)  

    dataframe['signal'] = dataframe['signal'].shift(1)
    
    return dataframe

create_signal(dataframe=df)

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma,signal
99,2023-01-20 01:00:00,1.08288,1.08366,1.08278,1.08323,15,1.080928,1.081667,1.080836,NaN
100,2023-01-20 02:00:00,1.08323,1.08395,1.08278,1.08302,15,1.081008,1.081666,1.080928,sell
101,2023-01-20 03:00:00,1.08305,1.08426,1.08271,1.08353,15,1.081102,1.081670,1.081008,sell
102,2023-01-20 04:00:00,1.08354,1.08431,1.08305,1.08356,15,1.081206,1.081675,1.081102,sell
103,2023-01-20 05:00:00,1.08356,1.08375,1.08315,1.08334,15,1.081282,1.081687,1.081206,sell
...,...,...,...,...,...,...,...,...,...,...
6495,2024-01-31 18:00:00,1.08603,1.08668,1.08438,1.08536,18,1.083146,1.083758,1.083051,sell
6496,2024-01-31 19:00:00,1.08535,1.08536,1.08410,1.08484,17,1.083250,1.083728,1.083146,sell
6497,2024-01-31 20:00:00,1.08485,1.08516,1.08429,1.08450,18,1.083330,1.083714,1.083250,sell
6498,2024-01-31 21:00:00,1.08449,1.08632,1.08149,1.08395,18,1.083388,1.083722,1.083330,sell


In [9]:
def create_ohlcv(dataframe):
  deals = dataframe.groupby((dataframe.signal != dataframe.signal.shift()).cumsum(), as_index= False).agg(
    open_time = ('time', 'first'),
    signal = ('signal', 'first'),
    open_price = ('open', 'first'),
    highest = ('high', 'max'),
    lowest = ('low', 'min')
  )

  safety = 0.97
  deals['open_price'] = round(deals['open_price'] * safety, 5)
  deals['close_price'] = deals['open_price'].shift(-1)
  deals['close_price'].fillna(value=dataframe['close'].iloc[-2], inplace=True)
  
  deals.dropna(inplace=True)
  
  return deals

deals = create_ohlcv(dataframe=df)

deals

,open_time,signal,open_price,highest,lowest,close_price
1,2023-01-20 02:00:00,sell,1.05073,1.08456,1.08197,1.05141
2,2023-01-20 10:00:00,buy,1.05141,1.08581,1.08014,1.05046
3,2023-01-20 19:00:00,sell,1.05046,1.08576,1.08285,1.05274
4,2023-01-20 22:00:00,buy,1.05274,1.09284,1.08342,1.05423
5,2023-01-30 05:00:00,sell,1.05423,1.10320,1.08007,1.06814
...,...,...,...,...,...,...
75,2024-01-02 13:00:00,sell,1.06759,1.10067,1.08757,1.06155
76,2024-01-08 15:00:00,buy,1.06155,1.09949,1.09096,1.05989
77,2024-01-16 04:00:00,sell,1.05989,1.09268,1.08435,1.05793
78,2024-01-22 05:00:00,buy,1.05793,1.09151,1.08207,1.05306


In [10]:
def calc_profit_dots(row):
    if row['signal'] == 'buy':
        val = row['close_price'] - row['open_price']
    elif row['signal'] == 'sell':
        val = row['open_price'] - row['close_price']
    else:
        val = 0
    return val

def calc_broker_digits(broker_digits):
    if broker_digits == 4:
        digits_factor = 1000
    elif broker_digits == 5:
        digits_factor = 10000
    return digits_factor

def apply_profit_dots(dataframe, function):
    dataframe['dots'] = dataframe.apply(function, axis=1)
    # dataframe['points'] = dataframe['dots'] * calc_broker_digits(broker_digits=5)
    dataframe['points'] = dataframe['dots'] 
    dataframe['type'] = np.where(dataframe['dots'] > 0, 'win', 'loss')
    # dataframe['mart_factor'] = (dataframe['profit_type'].groupby((dataframe['profit_type'] != dataframe['profit_type'].shift()).cumsum()).cumcount() + 1)
    dataframe.reset_index(drop=True, inplace=True)
    
    return dataframe

deals = apply_profit_dots(dataframe=deals, function=calc_profit_dots)

deals

,open_time,signal,open_price,highest,lowest,close_price,dots,points,type
0,2023-01-20 02:00:00,sell,1.05073,1.08456,1.08197,1.05141,-0.00068,-0.00068,loss
1,2023-01-20 10:00:00,buy,1.05141,1.08581,1.08014,1.05046,-0.00095,-0.00095,loss
2,2023-01-20 19:00:00,sell,1.05046,1.08576,1.08285,1.05274,-0.00228,-0.00228,loss
3,2023-01-20 22:00:00,buy,1.05274,1.09284,1.08342,1.05423,0.00149,0.00149,win
4,2023-01-30 05:00:00,sell,1.05423,1.10320,1.08007,1.06814,-0.01391,-0.01391,loss
...,...,...,...,...,...,...,...,...,...
74,2024-01-02 13:00:00,sell,1.06759,1.10067,1.08757,1.06155,0.00604,0.00604,win
75,2024-01-08 15:00:00,buy,1.06155,1.09949,1.09096,1.05989,-0.00166,-0.00166,loss
76,2024-01-16 04:00:00,sell,1.05989,1.09268,1.08435,1.05793,0.00196,0.00196,win
77,2024-01-22 05:00:00,buy,1.05793,1.09151,1.08207,1.05306,-0.00487,-0.00487,loss


In [11]:
def calc_martingale_factor(dataframe):
    
    dataframe['profit_code'] = np.where(dataframe['dots'] > 0, 0, 1)

    profit_code = list(deals['profit_code'])

    profit_code = pd.Series(profit_code)

    steps = (profit_code.cumsum() * profit_code).diff() # mask out the cumsum where we won [0 1 2 3 0 0 4 5 6 ... ]

    edges = steps < 0 # find where the cumsum steps down -> where we won
    diff_steps = steps[edges].diff() # find the length of each losing streak
    diff_steps[steps[edges].index[0]] = steps[edges][:1] # fix length of the first run which in now NaN
    toss = profit_code.copy() # get a copy of the profit_code series
    toss[edges] = diff_steps # insert the length of the losing streaks into the copy of the profit_code results
    martingale_factor = 2 ** (toss).cumsum() # compute the wagers

    result = pd.DataFrame({
        'profit_code': profit_code,
        'toss': toss,
        'runs': toss.cumsum(),
        'martingale': martingale_factor
        })
    
    result['profit_toss'] = np.where(result['profit_code'] == 0, 'win', 'loss')
    
    result['martingale'] = result['martingale'].shift(periods=1, fill_value=1)
    
    # drop profit_code from dataframe
    dataframe = dataframe.drop('profit_code', axis=1) 
    
    # add mart_profit to the dataframe
    dataframe['martingale']= result['martingale']
    return dataframe

deals = calc_martingale_factor(dataframe=deals)

deals

,open_time,signal,open_price,highest,lowest,close_price,dots,points,type,martingale
0,2023-01-20 02:00:00,sell,1.05073,1.08456,1.08197,1.05141,-0.00068,-0.00068,loss,1
1,2023-01-20 10:00:00,buy,1.05141,1.08581,1.08014,1.05046,-0.00095,-0.00095,loss,2
2,2023-01-20 19:00:00,sell,1.05046,1.08576,1.08285,1.05274,-0.00228,-0.00228,loss,4
3,2023-01-20 22:00:00,buy,1.05274,1.09284,1.08342,1.05423,0.00149,0.00149,win,8
4,2023-01-30 05:00:00,sell,1.05423,1.10320,1.08007,1.06814,-0.01391,-0.01391,loss,1
...,...,...,...,...,...,...,...,...,...,...
74,2024-01-02 13:00:00,sell,1.06759,1.10067,1.08757,1.06155,0.00604,0.00604,win,1
75,2024-01-08 15:00:00,buy,1.06155,1.09949,1.09096,1.05989,-0.00166,-0.00166,loss,1
76,2024-01-16 04:00:00,sell,1.05989,1.09268,1.08435,1.05793,0.00196,0.00196,win,2
77,2024-01-22 05:00:00,buy,1.05793,1.09151,1.08207,1.05306,-0.00487,-0.00487,loss,1


In [12]:
# Lot Size:
# Standard => Volume = 100,000 => Lot = 1.00 => pip Value = 10.00 USD
# Mini Lot => Volume = 10,000 => Lot = 0.10 => pip Value = 1.00 USD
# Micro Lot => Volume = 1,000 => Lot = 0.01 => pip Value = 0.10 USD
# Nano Lot => Volume = 100 => Lot = 0.0001 => pip Value = 10 USD

deposit = 1000
contract = 10000000
min_lot= 0.01
volume = contract * min_lot


def create_profit_values(dataframe):
    dataframe['lot'] = dataframe['martingale'] * min_lot
    # dataframe['profit'] = dataframe['dots'] * dataframe['lot'] * volume
    dataframe['profit'] = dataframe['dots'] * dataframe['lot'] * 10

    dataframe['pnl_points'] = dataframe['points'].cumsum()
    dataframe['pnl_value'] = dataframe['profit'].cumsum() + deposit
    # dataframe['fast_sma_period'] = fast_sma_period
    # dataframe['slow_sma_period'] = slow_sma_period
    return dataframe

deals = create_profit_values(dataframe=deals)

deals

,open_time,signal,open_price,highest,lowest,close_price,dots,points,type,martingale,lot,profit,pnl_points,pnl_value
0,2023-01-20 02:00:00,sell,1.05073,1.08456,1.08197,1.05141,-0.00068,-0.00068,loss,1,0.01,-0.000068,-0.00068,999.999932
1,2023-01-20 10:00:00,buy,1.05141,1.08581,1.08014,1.05046,-0.00095,-0.00095,loss,2,0.02,-0.000190,-0.00163,999.999742
2,2023-01-20 19:00:00,sell,1.05046,1.08576,1.08285,1.05274,-0.00228,-0.00228,loss,4,0.04,-0.000912,-0.00391,999.998830
3,2023-01-20 22:00:00,buy,1.05274,1.09284,1.08342,1.05423,0.00149,0.00149,win,8,0.08,0.001192,-0.00242,1000.000022
4,2023-01-30 05:00:00,sell,1.05423,1.10320,1.08007,1.06814,-0.01391,-0.01391,loss,1,0.01,-0.001391,-0.01633,999.998631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2024-01-02 13:00:00,sell,1.06759,1.10067,1.08757,1.06155,0.00604,0.00604,win,1,0.01,0.000604,-0.10552,1000.279184
75,2024-01-08 15:00:00,buy,1.06155,1.09949,1.09096,1.05989,-0.00166,-0.00166,loss,1,0.01,-0.000166,-0.10718,1000.279018
76,2024-01-16 04:00:00,sell,1.05989,1.09268,1.08435,1.05793,0.00196,0.00196,win,2,0.02,0.000392,-0.10522,1000.279410
77,2024-01-22 05:00:00,buy,1.05793,1.09151,1.08207,1.05306,-0.00487,-0.00487,loss,1,0.01,-0.000487,-0.11009,1000.278923


In [13]:
def calc_max_tp_sl(dataframe):
    # dataframe['max_tp_points'] = np.where(dataframe['signal'] == 'buy', (dataframe['highest'] - dataframe['open_price']) * calc_broker_digits(broker_digits=5), 
    #                                (dataframe['open_price'] - dataframe['lowest']) * calc_broker_digits(broker_digits=5))
    # dataframe['max_sl_points'] = np.where(dataframe['signal'] == 'buy', (dataframe['open_price'] - dataframe['lowest'])  * calc_broker_digits(broker_digits=5), 
    #                                (dataframe['highest'] - dataframe['open_price']) * calc_broker_digits(broker_digits=5))
    
    dataframe['max_tp_points'] = np.where(dataframe['signal'] == 'buy', (dataframe['highest'] - dataframe['open_price']), 
                                   (dataframe['open_price'] - dataframe['lowest']) )
    dataframe['max_sl_points'] = np.where(dataframe['signal'] == 'buy', (dataframe['open_price'] - dataframe['lowest']) , 
                                   (dataframe['highest'] - dataframe['open_price']))
    return dataframe

deals = calc_max_tp_sl(dataframe=deals)

# pd.set_option('display.max_rows', None)
deals

,open_time,signal,open_price,highest,lowest,close_price,dots,points,type,martingale,lot,profit,pnl_points,pnl_value,max_tp_points,max_sl_points
0,2023-01-20 02:00:00,sell,1.05073,1.08456,1.08197,1.05141,-0.00068,-0.00068,loss,1,0.01,-0.000068,-0.00068,999.999932,-0.03124,0.03383
1,2023-01-20 10:00:00,buy,1.05141,1.08581,1.08014,1.05046,-0.00095,-0.00095,loss,2,0.02,-0.000190,-0.00163,999.999742,0.03440,-0.02873
2,2023-01-20 19:00:00,sell,1.05046,1.08576,1.08285,1.05274,-0.00228,-0.00228,loss,4,0.04,-0.000912,-0.00391,999.998830,-0.03239,0.03530
3,2023-01-20 22:00:00,buy,1.05274,1.09284,1.08342,1.05423,0.00149,0.00149,win,8,0.08,0.001192,-0.00242,1000.000022,0.04010,-0.03068
4,2023-01-30 05:00:00,sell,1.05423,1.10320,1.08007,1.06814,-0.01391,-0.01391,loss,1,0.01,-0.001391,-0.01633,999.998631,-0.02584,0.04897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2024-01-02 13:00:00,sell,1.06759,1.10067,1.08757,1.06155,0.00604,0.00604,win,1,0.01,0.000604,-0.10552,1000.279184,-0.01998,0.03308
75,2024-01-08 15:00:00,buy,1.06155,1.09949,1.09096,1.05989,-0.00166,-0.00166,loss,1,0.01,-0.000166,-0.10718,1000.279018,0.03794,-0.02941
76,2024-01-16 04:00:00,sell,1.05989,1.09268,1.08435,1.05793,0.00196,0.00196,win,2,0.02,0.000392,-0.10522,1000.279410,-0.02446,0.03279
77,2024-01-22 05:00:00,buy,1.05793,1.09151,1.08207,1.05306,-0.00487,-0.00487,loss,1,0.01,-0.000487,-0.11009,1000.278923,0.03358,-0.02414


In [14]:
def calc_result(dataframe):
    
    
    
    # find balance
    balance = round(dataframe['pnl_value'].iloc[-1], 2)
    print('balance = ', balance)

    # find total net profit
    total_net_profit = round(dataframe['profit'].sum(), 2)  # Use in backtest Factors
    print('total net profit = ', total_net_profit)

    # find gross profit
    gross_profit = round(sum(dataframe[dataframe['type']=='win']['profit']), 2)
    print('gross profit = ', gross_profit)

    # find gross loss
    gross_loss = round(sum(dataframe[dataframe['type']=='loss']['profit']), 2)
    print('gross loss = ', gross_loss)

    # find profit factor
    profit_factor = abs(round((gross_profit / gross_loss), 2))
    print('profit factor = ', profit_factor)
    
    # find largest profit trade
    largest_profit_trade = round(dataframe['profit'].max(), 2)
    print('largest profit trade = ', largest_profit_trade)
    
    # find largest profit trade
    df2=dataframe.loc[dataframe['profit'] == dataframe['profit'].max(), 'open_time'].item()
    print('largest profit trade open time = ', df2)

    # find largest loss trade
    largest_loss_trade = round(dataframe['profit'].min(), 2)
    print('largest loss trade = ', largest_loss_trade)

    # find total trades
    total_trades = len(dataframe)
    print('total trades = ', total_trades)

    # find total count of SELL trades
    count_sell_trades = dataframe['signal'].value_counts()['sell']
    print('count SELL trades = ', count_sell_trades)

    # find total count of BUY trades
    count_buy_trades = dataframe['signal'].value_counts()['buy']
    print('count BUY trades = ', count_buy_trades)

    # find total count of WIN trades
    count_win_trades = dataframe['type'].value_counts()['win']
    print('count WIN trades = ', count_win_trades)
    
    # find total count of LOSS trades
    count_loss_trades = dataframe['type'].value_counts()['loss']
    print('count LOSS trades = ', count_loss_trades)

    # select rows by value
    # avg_win = deals_group.loc[deals_group['profit_type'] == 'win']
    # print('avg_win' , avg_win)

    # find average value of WIN Trades
    ave_win_value = round(dataframe.loc[dataframe['type'] == 'win']['profit'].mean() , 2)
    print('ave_win_value = ' , ave_win_value)

    # find average value of LOSS Trades
    ave_loss_value = round(dataframe.loc[dataframe['type'] == 'loss']['profit'].mean(), 2)
    print('ave_loss_value = ' , ave_loss_value)
    
    # find max take profit points
    max_tp_points = round(dataframe['max_tp_points'].max(), 2)
    print('max take profit points = ', max_tp_points)
 
    # find average take profit points
    avg_tp_points = round(dataframe['max_tp_points'].mean(), 2)
    print('average take profit points = ', avg_tp_points)   
    
    # find min take profit points
    min_tp_points = round(dataframe['max_tp_points'].min(), 2)
    print('min take profit points = ', min_tp_points)   
    
    # find max stop loss points
    max_sl_points = round(dataframe['max_sl_points'].max(), 2)
    print('max stop loss points = ', max_sl_points)
    
    # find average stop loss points
    avg_sl_points = round(dataframe['max_sl_points'].mean(), 2)
    print('average stop loss points = ', avg_sl_points) 
    
    # find min stop loss points
    min_sl_points = round(dataframe['max_sl_points'].min(), 2)
    print('min stop loss points = ', min_sl_points)
    
    # find max take profit points open time
    df3=dataframe.loc[dataframe['max_tp_points'] == dataframe['max_tp_points'].max(), 'open_time'].item()
    print('max take profit points open time = ', df3)
    
    # find max stop loss points open time
    df4=dataframe.loc[dataframe['max_sl_points'] == dataframe['max_sl_points'].max(), 'open_time'].item()
    print('max stop loss points open time = ', df4)
    
    # Collect consecutive Profit / Loss (Factors)

    # find consecutive Profit / Loss
    positions_group = dataframe.groupby((dataframe.type != dataframe.type.shift()).cumsum()).agg(
    type = ('type', 'first'),
    grand_profit = ('profit', 'sum'),
    grand_points = ('points', 'sum'),
    count = ('type', 'count')
    ).reset_index(drop=True)

    print('count trades groups = ')
    print(positions_group)

    # find largest consecutive WIN trade
    largest_cons_win = round(positions_group['grand_profit'].max(), 2)
    print('largest consecutive WIN trade = ', largest_cons_win)

    # find largest consecutive WIN trade (count)
    count_largest_win = positions_group.loc[positions_group['grand_profit'].idxmax(), 'count']
    print('largest consecutive WIN trade (count) = ', count_largest_win)

    # find largest consecutive LOSS trade
    largest_cons_loss = round(positions_group['grand_profit'].min(), 2)
    print('largest consecutive LOSS trade = ', largest_cons_loss)

    # find largest consecutive LOSS trade (count)
    count_largest_loss = positions_group.loc[positions_group['grand_profit'].idxmin(), 'count']
    print('largest consecutive LOSS trade (count) = ', count_largest_loss)
    
    result = {
        'fast_sma': [fast_sma_period], # fast SMA
        'slow_sma': [slow_sma_period], # slow SMA
        'deposit': [deposit], # deposit
        'balance': [balance], # balance
        'pnl': [total_net_profit],
        'gross_profit': [gross_profit],
        'gross_loss': [gross_loss],
        'profit_factor': [profit_factor],
        'max_win': [largest_profit_trade],
        'max_loss': [largest_loss_trade],
        'total_trades': [total_trades],
        'count_sell': [count_sell_trades],
        'count_buy': [count_buy_trades],
        'count_win': [count_win_trades],
        'count_loss': [count_loss_trades],
        'ave_win': [ave_win_value],
        'ave_loss': [ave_loss_value],
        'max_tp_points': [max_tp_points],
        'avg_tp_points': [avg_tp_points],
        'min_tp_points': [min_tp_points],
        'max_sl_points': [max_sl_points],
        'avg_sl_points': [avg_sl_points],
        'min_sl_points': [min_sl_points],
        'max_cons_win': [largest_cons_win],
        'count_max_cons_win': [count_largest_win],
        'max_cons_loss': [largest_cons_loss],
        'count_max_cons_loss': [count_largest_loss],
    }
    result_df = pd.DataFrame.from_dict(result)
    return result_df

result_df = calc_result(dataframe=deals)

result_df

balance =  1000.27
total net profit =  0.27
gross profit =  0.58
gross loss =  -0.31
profit factor =  1.87
largest profit trade =  0.29
largest profit trade open time =  2023-07-10 02:00:00
largest loss trade =  -0.07
total trades =  79
count SELL trades =  40
count BUY trades =  39
count WIN trades =  23
count LOSS trades =  56
ave_win_value =  0.03
ave_loss_value =  -0.01
max take profit points =  0.06
average take profit points =  0.01
min take profit points =  -0.03
max stop loss points =  0.05
average stop loss points =  0.01
min stop loss points =  -0.03
max take profit points open time =  2023-07-10 02:00:00
max stop loss points open time =  2023-01-30 05:00:00
count trades groups = 
    type  grand_profit  grand_points  count
0   loss     -0.001170      -0.00391      3
1    win      0.001192       0.00149      1
2   loss     -0.005765      -0.03578      2
3    win      0.002664       0.00666      1
4   loss     -0.008394      -0.01440      5
5    win      0.017410       0.00949

,fast_sma,slow_sma,deposit,balance,pnl,gross_profit,gross_loss,profit_factor,max_win,max_loss,...,max_tp_points,avg_tp_points,min_tp_points,max_sl_points,avg_sl_points,min_sl_points,max_cons_win,count_max_cons_win,max_cons_loss,count_max_cons_loss
0,50,100,1000,1000.27,0.27,0.58,-0.31,1.87,0.29,-0.07,...,0.06,0.01,-0.03,0.05,0.01,-0.03,0.29,3,-0.1,7


In [15]:
px.line(deals, x='open_time', y='pnl_value')

In [16]:
# def find_crossover(fast_sma, prev_fast_sma, slow_sma):
    
#     if fast_sma > slow_sma and prev_fast_sma < slow_sma:
#         return 'bullish crossover'
#     elif fast_sma < slow_sma and prev_fast_sma > slow_sma:
#         return 'bearish crossover'
    
#     return None


# df['crossover'] = np.vectorize(find_crossover)(df['fast_sma'], df['prev_fast_sma'], df['slow_sma'])

# # df.dropna(inplace=True)
# # signal = df[df['crossover'] == 'bullish crossover'].copy() # => Original
# signal = df[(df['crossover'] == 'bullish crossover') | (df['crossover'] == 'bearish crossover')].copy()

# signal

In [17]:
# # visualize close price
# fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

# for i, row in signal.iterrows():
#     fig.add_vline(x=row.time)
    
# fig.show()

In [18]:
# # visualize close price
# fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

# for i, row in signal.iterrows():
#     fig.add_vline(x=row.time)
    
# for i, row in deals[deals['status'] == 'closed'].iterrows():
    
#     if row.profit > 0:
#         fig.add_shape(type="line",
#             x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
#             line=dict(color="Green",width=3)
#                      )
                      
#     elif row.profit < 0:
#         fig.add_shape(type="line",
#             x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
#             line=dict(color="Red",width=3)
#                       )

    
# fig.show()

In [19]:
# Create Summary DataFrame
summary = pd.DataFrame(columns=['fast_sma',
                                'slow_sma',
                                
                                'deposit',  # deposit
                                'balance',  # balance
                                
                                'net_profit', # total net profit
                                
                                'gross_profit', # sum of win trades
                                'gross_loss', # sum of fail trades
                                'profit_factor', # ratio (%) => (gross profit / gross loss) => one means (profit = loss)
                                
                                'abs_drawdown', # the largest loss is lower than the initial deposit value
                                'max_drawdown', # maximal loss of the local maximum in the deposit currency and in percents of the deposit
                                'rel_drawdown', #  the maximal loss in percents of the maximum equity value and its corresponding money value
                                
                                'total_trades', # count total trades
                                'total_buy', # count total BUY trades
                                'total_sell', # count total SELL trades
                                
                                'max_win', # Largest profit trade
                                'max_fail', # Largest loss trade
                                
                                'min_lot', # initial lot
                                'lot_multiplier',
                                'max_lot', # highest lot
                                
                                'F',
                                'G'])

# balance =  9971.76
# total net profit =  -28.24
# gross profit =  5.43
# gross loss =  -33.67
# profit factor =  0.16
# largest profit trade =  5.2
# largest loss trade =  -2.2
# total trades =  65
# count SELL trades =  24
# count BUY trades =  41
# count WIN trades =  5
# count LOSS trades =  60
# ave_win_value =  1.09
# ave_loss_value =  -0.56

summary

,fast_sma,slow_sma,deposit,balance,net_profit,gross_profit,gross_loss,profit_factor,abs_drawdown,max_drawdown,...,total_trades,total_buy,total_sell,max_win,max_fail,min_lot,lot_multiplier,max_lot,F,G


In [20]:
def get_broker_data():
    # Get QTY of Symbols Available within the Broker
    symbol_qty = mt5.symbols_total()
    print("symbol_qty = ", symbol_qty)

    # Get Lit of all available symbols
    symbols = mt5.symbols_get()
    
    for symbol in symbols:
        print("symbols = ", symbol)
    
get_broker_data()

symbol_qty =  1099
symbols =  SymbolInfo(custom=False, chart_mode=0, select=False, visible=False, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=0, volumehigh=0, volumelow=0, time=0, digits=5, spread=0, spread_float=True, ticks_bookdepth=10, trade_calc_mode=0, trade_mode=4, start_time=0, expiration_time=0, trade_stops_level=55, trade_freeze_level=5, trade_exemode=2, swap_mode=1, swap_rollover3days=4, margin_hedged_use_leg=False, expiration_mode=15, filling_mode=1, order_mode=127, order_gtc_mode=0, ...)
symbols =  SymbolInfo(custom=False, chart_mode=0, select=True, visible=False, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=0, volumehigh=0, volumelow=0, time=1706741511, digits=5, spread=201, spread_float=True, ticks_bookdepth=10, trade_calc_mode=0, trade_mode=4, start_time=0, expiration_time=0, trade_stops_level=0, trade_freeze_level=0, trade_exemode=2, swap_mode=1, swap_rollover3days=3, margin_hedged_use_leg=False, expiration_mode=15, filli